# Flow Run Management in Azure

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed PromptFlow SDK


**Learning Objectives** - By the end of this tutorial, you should be able to:
- create run with remote data
- create run which references another runs inputs
- manage runs via run.yaml
- serverless runtime


**Motivations** - This guide will walk you through cloud run management abilities.

## 0. Install dependent packages

In [ ]:
%pip install -r ../../requirements.txt

## 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import Data
from azure.core.exceptions import ResourceNotFoundError

from promptflow.azure import PFClient
from promptflow.entities import Run

### 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

### 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)
pf = PFClient.from_config(credential=credential)

### 1.4 Create necessary connections
Connection helps securely store and manage secret keys or other sensitive credentials required for interacting with LLM and other external tools for example Azure Content Safety.

In this notebook, we will use flow `web-classification` which uses connection `azure_open_ai_connection` inside, we need to set up the connection if we haven't added it before.

Prepare your Azure OpenAI resource follow this [instruction](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal) and get your `api_key` if you don't have one.

Please go to [workspace portal](https://ml.azure.com/), click `Prompt flow` -> `Connections` -> `Create`, then follow the instruction to create your own connections. 
Learn more on [connections](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/concept-connections?view=azureml-api-2).

### 1.5 Create runtime
Prompt flow’s runtime provides the computing resources required for the application to run. 

In this notebook, you can follow this [instruction](https://learn.microsoft.com/en-us/azure/machine-learning/prompt-flow/how-to-create-manage-runtime?view=azureml-api-2) to create a runtime in workspace then use it to submit flow runs.

In [ ]:
runtime = "demo-mir"  # TODO replace with your runtime name

## 2. Create run with remote data

### 2.1 Create or update remote data

In [ ]:
data_name, data_version = "flow_run_test_data", "1"

try:
    data = ml_client.data.get(name=data_name, version=data_version)
except ResourceNotFoundError:
    data = Data(
        name=data_name,
        version=data_version,
        path=f"../../flows/standard/web-classification/data.jsonl",
        type="uri_file",
    )
    data = ml_client.data.create_or_update(data)

### 2.2 Prepare remote data id

In [ ]:
data_id = f"azureml:{data.name}:{data.version}"
print(data_id)

### 2.3 Create a flow run with remote data

In [ ]:
# create run
run = Run(
    # local flow file
    flow="../../flows/standard/web-classification",
    # remote data
    data=data_id,
)

base_run = pf.runs.create_or_update(
    run=run,
    runtime=runtime,
)

### 2.4 Stream the flow run to make sure it runs successfully

In [ ]:
pf.runs.stream(base_run)

## 3 Create a flow run which use an existing run's inputs data

In [ ]:
run = Run(
    # local flow file
    flow="../../flows/standard/web-classification",
    # run name
    run=run,
    column_mapping={
        # reference another run's inputs
        "url": "${run.inputs.url}",
        "answer": "${run.inputs.answer}",
        "evidence": "${run.inputs.evidence}",
    },
)

base_run = pf.runs.create_or_update(
    run=run,
    runtime=runtime,
)